In [1]:
import numpy as np

def g(player):
    phi = player.phi
    return 1/(np.sqrt(1+3*(phi/(np.pi))**2))

def E(player1,player2):
    mu = player1.mu
    mu2 = player2.mu
    return 1/(1+np.exp(g(player2)*(mu2-mu)))

def g_p(player1,player2):
    phi = np.sqrt(player1.phi**2+player2.phi**2)
    return 1/(np.sqrt(1+3*(phi/(np.pi))**2))


def E_p(player1,player2):
    mu = player1.mu
    mu2 = player2.mu
    return 1/(1+np.exp(g_p(player1,player2)*(mu2-mu)))

class Player:
    tau = 1/np.sqrt(2) #this is a constant that can be fine-tuned.
    def __init__(self, name, mu=0, phi=2, rho=0.06):
        self.name = name
        self.mu = mu
        self.phi = phi
        self.rho = rho
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.name == other.name
        else:
            return False

    def estimate_variance(self, games):
        v_sum = 0
        for game in games:
            if self == game.player1:
                E_iter = E(self,game.player2)
                v_sum = v_sum + (g(game.player2))**2*(E_iter-E_iter**2)
            if self == game.player2:
                E_iter = E(self,game.player1)
                v_sum = v_sum + (g(game.player1))**2*(E_iter-E_iter**2)
        if v_sum!=0:
            self.est_variance = 1/v_sum
        else:
            self.est_variance = -1

    def estimate_Delta(self, games):
        d_sum = 0
        for game in games:
            if self == game.player1:
                d_sum = d_sum + g(game.player2)*(game.score-E(self,game.player2))
            if self == game.player2:
                d_sum = d_sum + g(game.player1)*(1-game.score-E(self,game.player1))
        self.est_Delta = self.est_variance*d_sum
    
    def _f(self,x):
        ex = np.exp(x)
        a = np.log(self.phi**2)
        return (ex*(self.est_Delta**2-self.phi**2-self.est_variance-ex)/(2*(self.phi**2+self.est_variance+ex)**2)
                - (x - a)/self.tau**2)
    
    def compute_volatility(self):
        #basically the Illinois algorithm lifted from the glicko2 pdf. Illini!
        if self.est_variance!=-1:
            a = np.log(self.phi**2)
            A = a
            if self.est_Delta**2>(self.phi**2+self.est_variance):
                B = np.log(self.est_Delta**2-self.phi**2-self.est_variance)
            else:
                k = 1
                while(self._f(a-k*self.tau)<0):
                    k = k+1
                B = a-k*self.tau
            fA = self._f(A)
            fB = self._f(B)
            while(abs(B-A)>1e-6):
                C = A + (A-B)*fA/(fB-fA)
                fC = self._f(C)
                if fC*fB<0:
                    A = B
                    fA = fB
                else:
                    fA = fA/2    
                B = C
                fB = fC
            self.rho = np.exp(A/2)
            phi_d = np.sqrt(self.phi**2+self.rho**2)
            self.phi = 1/np.sqrt(1/phi_d**2+1/self.est_variance)
            self.mu = self.mu + self.phi**2/self.est_variance*self.est_Delta

class Game:
    def __init__(self, player1, player2, score):
        '''score=1 means player1 wins, score=0 means player2 wins, score=0.5 means draw'''
        self.player1 = player1
        self.player2 = player2
        self.score = score

        

In [2]:
BOS = Player('BOS')
FLA = Player('FLA')
HOU = Player('HOU')
LDN = Player('LDN')
NYE = Player('NYE')
PHI = Player('PHI')
DAL = Player('DAL')
GLA = Player('GLA')
VAL = Player('VAL')
SFS = Player('SFS')
SEO = Player('SEO')
SHD = Player('SHD')
players = [BOS, FLA, HOU, LDN, NYE, PHI, DAL, GLA, VAL, SFS, SEO, SHD]

In [3]:
games = [
    Game(FLA,SFS,0), Game(FLA,SFS,0), Game(FLA,SFS,0), Game(FLA,SFS,1), 
    Game(VAL,SFS,1), Game(VAL,SFS,1), Game(VAL,SFS,1), Game(VAL,SFS,0), Game(VAL,SFS,0), 
    Game(SEO,SHD,1), Game(SEO,SHD,1), Game(SEO,SHD,1), Game(SEO,SHD,1), 
    Game(GLA,LDN,1), Game(GLA,LDN,1), Game(GLA,LDN,1), Game(GLA,LDN,0), Game(GLA,LDN,0), 
    Game(DAL,HOU,1), Game(DAL,HOU,1), Game(DAL,HOU,1), Game(DAL,HOU,0), Game(DAL,HOU,0), 
    Game(NYE,BOS,1), Game(NYE,BOS,1), Game(NYE,BOS,1), Game(NYE,BOS,0), 
    Game(SFS,LDN,0), Game(SFS,LDN,0), Game(SFS,LDN,0), Game(SFS,LDN,0), 
    Game(SEO,HOU,1), Game(SEO,HOU,1), Game(SEO,HOU,0.5), Game(SEO,HOU,0), 
    Game(SHD,BOS,1), Game(SHD,BOS,1), Game(SHD,BOS,0), Game(SHD,BOS,0), Game(SHD,BOS,0), 
    Game(VAL,GLA,1), Game(VAL,GLA,1), Game(VAL,GLA,1), Game(VAL,GLA,0), 
    Game(FLA,DAL,1), Game(FLA,DAL,0), Game(FLA,DAL,0), Game(FLA,DAL,0), 
    Game(SEO,NYE,1), Game(SEO,NYE,1), Game(SEO,NYE,1), Game(SEO,NYE,0), 
]
for player in players:
    player.estimate_variance(games)
    player.estimate_Delta(games)
    player.compute_volatility()

for player in players:
    print('{0}: {1:.3f}±{2:.3f}'.format(player.name,player.mu,2*player.phi))

BOS: -0.293±1.868
FLA: -1.301±1.968
HOU: -0.586±1.868
LDN: 0.879±1.868
NYE: -0.009±1.715
PHI: 0.000±4.000
DAL: -0.093±1.570
GLA: 0.418±1.670
VAL: 1.063±1.638
SFS: -0.261±1.346
SEO: 1.043±1.368
SHD: -0.977±1.491


In [4]:
games = [
    Game(SFS,VAL,0), Game(SFS,VAL,0), Game(SFS,VAL,0), Game(SFS,VAL,0), 
    Game(SHD,GLA,0), Game(SHD,GLA,0), Game(SHD,GLA,0), Game(SHD,GLA,0), 
    Game(DAL,SEO,1), Game(DAL,SEO,0), Game(DAL,SEO,0), Game(DAL,SEO,0.5), 
    Game(LDN,FLA,0), Game(LDN,FLA,1), Game(LDN,FLA,1), Game(LDN,FLA,1), 
    Game(PHI,HOU,1), Game(PHI,HOU,0), Game(PHI,HOU,1), Game(PHI,HOU,0), Game(PHI,HOU,1), 
    Game(BOS,NYE,0), Game(BOS,NYE,1), Game(BOS,NYE,0), Game(BOS,NYE,0), 
    Game(VAL,DAL,1), Game(VAL,DAL,1), Game(VAL,DAL,1), Game(VAL,DAL,0.5), 
    Game(FLA,BOS,0), Game(FLA,BOS,0), Game(FLA,BOS,0), Game(FLA,BOS,0), 
    Game(SFS,SHD,1), Game(SFS,SHD,1), Game(SFS,SHD,1), Game(SFS,SHD,0), 
    Game(LDN,PHI,1), Game(LDN,PHI,1), Game(LDN,PHI,1), Game(LDN,PHI,1), 
    Game(NYE,HOU,1), Game(NYE,HOU,1), Game(NYE,HOU,1), Game(NYE,HOU,0), 
    Game(SEO,GLA,1), Game(SEO,GLA,1), Game(SEO,GLA,1), Game(SEO,GLA,1), 
]
for player in players:
    player.estimate_variance(games)
    player.estimate_Delta(games)
    player.compute_volatility()

for player in players:
    print('{0}: {1:.3f}±{2:.3f}'.format(player.name,player.mu,2*player.phi))

BOS: -0.120±1.400
FLA: -1.620±1.615
HOU: -0.824±1.450
LDN: 1.177±1.765
NYE: 0.462±1.313
PHI: -0.778±1.541
DAL: -0.113±1.365
GLA: 0.164±1.371
VAL: 1.708±1.405
SFS: -0.338±1.314
SEO: 1.317±1.264
SHD: -1.541±1.293


In [5]:
x=8*(E_p(SFS,PHI)-0.5)
print('SFS v PHI: {:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))
x=8*(E_p(FLA,SEO)-0.5)
print('FLA v SEO: {:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))
x=8*(E_p(HOU,SHD)-0.5)
print('HOU v SHD: {:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))

SFS v PHI: 0.758, 2.4-1.6
FLA v SEO: -3.424, 0.3-3.7
HOU v SHD: 1.224, 2.6-1.4


In [6]:
games = [
    Game(SFS,PHI,1), Game(SFS,PHI,0), Game(SFS,PHI,0), Game(SFS,PHI,0.5), 
    Game(SEO,FLA,1), Game(SEO,FLA,1), Game(SEO,FLA,1), Game(SEO,FLA,1), 
    Game(HOU,SHD,1), Game(HOU,SHD,1), Game(HOU,SHD,1), Game(HOU,SHD,1), 
    Game(DAL,HOU,0), Game(DAL,HOU,0), Game(DAL,HOU,0), Game(DAL,HOU,0), 
    Game(NYE,VAL,1), Game(NYE,VAL,1), Game(NYE,VAL,1), Game(NYE,VAL,0), 
    Game(PHI,GLA,1), Game(PHI,GLA,1), Game(PHI,GLA,0), Game(PHI,GLA,0), Game(PHI,GLA,0), 
]
for player in players:
    player.estimate_variance(games)
    player.estimate_Delta(games)
    player.compute_volatility()

for player in players:
    print('{0}: {1:.3f}±{2:.3f}'.format(player.name,player.mu,2*player.phi))

BOS: -0.120±1.400
FLA: -1.848±2.028
HOU: 0.709±1.280
LDN: 1.177±1.765
NYE: 1.574±1.526
PHI: -0.261±1.236
DAL: -0.760±1.475
GLA: 0.166±1.357
VAL: 1.154±1.463
SFS: -0.532±1.386
SEO: 1.462±1.673
SHD: -1.802±1.608


In [11]:
ratings={}
for player in players:
    ratings[player.name] = (player.mu*173.7178+1500,2*player.phi*173.7178)

iter_keys = list(ratings.keys())
while len(iter_keys)>0:
    max = 0
    for key in iter_keys:
        if ratings[key][0]>max:
            max=ratings[key][0]
            max_key = key
    iter_keys.remove(max_key)
    print('{0}|{1[0]:.0f}±{1[1]:.0f}'.format(max_key,ratings[max_key]))

NYE|1773±265
SEO|1754±291
LDN|1704±307
VAL|1700±254
HOU|1623±222
GLA|1529±236
BOS|1479±243
PHI|1455±215
SFS|1408±241
DAL|1368±256
SHD|1187±279
FLA|1179±352


In [12]:
x=8*(E_p(SEO,BOS)-0.5)
print('SEO v BOS|{:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))
x=8*(E_p(SHD,FLA)-0.5)
print('SHD v FLA|{:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))
x=8*(E_p(LDN,DAL)-0.5)
print('LDN v DAL|{:.3f}, {:.1f}-{:.1f}'.format(x,2+x/2,2-x/2))

SEO v BOS|2.361, 3.2-0.8
SHD v FLA|0.076, 2.0-2.0
LDN v DAL|2.696, 3.3-0.7
